Import needed libraries

In [1]:
import pandas as pd

Read data

In [2]:
# These links must not be changed and you can access data from them at anytime
# Just make sure to have quite stable internet connection
KAGGLE_DATA_PATH = 'hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/kaggle_data.feather'
ZENODO_MENTALHEALTH_DATA_PATH = 'hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/zenodo_mentalhealth_data.feather'
ZENODO_NOMENTALHEALTH_DATA_PATH = 'hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/zenodo_nomentalhealth_data.feather'

In [ ]:
kaggle_df = pd.read_feather(KAGGLE_DATA_PATH)

In [4]:
kaggle_df.sample(n=20, random_state=123)

,text,label
557,dont even know say girlfriend last night party...,0
17016,wish could end all tired living fake life arou...,1
4347,miss someone miss someone isnt valentines day ...,0
15336,pointthats really dont get it whats point livi...,1
25878,impossible avoid punishment hispanic parents s...,0
11100,anyone else weird eye thing literally person s...,0
27205,well guess im waiting right time placei though...,1
9932,update supposed nowabout month ago posted want...,1
14823,need fucking help please know put it im used a...,1
2464,think related roommate last name parents signs...,0


In [5]:
zenodo_mentalhealth_df = pd.read_feather(ZENODO_MENTALHEALTH_DATA_PATH)

In [6]:
zenodo_mentalhealth_df.sample(n=20, random_state=123)

,text,label
27357,Tonight some asshole went and downvoted a bunc...,depression
536,Hello all! Helpful resource here Have been dea...,healthanxiety
2702,I feel stable until I get into a relationship ...,bpd
894,"When high school ends, I might just too Life i...",suicidewatch
21812,I feel like I'm losing myself I found this sub...,depression
13853,"Tw, vent/letter to psychiatrist For background...",suicidewatch
8587,Why is it so easy for everyone but not me? As ...,depression
19649,Sometimes I don’t know what to say when people...,depression
14459,Useless Days feel weird for some reason this w...,suicidewatch
2090,I was working overtime. None of my friends cal...,lonely


In [7]:
zenodo_nomentalhealth_df = pd.read_feather(ZENODO_NOMENTALHEALTH_DATA_PATH)

In [8]:
zenodo_nomentalhealth_df.sample(n=20, random_state=123)

,text,label
27127,I haven't had a working toilet in over a week ...,legaladvice
2810,North american arms Hello all! I am looking to...,guns
48374,Builder next door has dumped rubbish on my lan...,legaladvice
4811,The Importance of Eating Big I’m 24 and I’ve b...,fitness
39256,Some background to my problem I'm 14 male. My ...,legaladvice
2678,Is biting at daycare normal? My toddler came h...,parenting
17838,Help Understanding Retirement Benefits: Public...,personalfinance
1616,Grad School Stipend and Taxes My partner is re...,personalfinance
1764,Question: does the ignition on the back of a t...,guns
26325,I just died and god tells me: “now I’m gonna r...,jokes


In [ ]:
normal_df = kaggle_df[kaggle_df["label"]==0]
normal_df["label"] = normal_df["label"].replace(0, "normal")
#normal_df = normal_df.sample(n=7000, random_state=123)

In [ ]:
data = pd.concat([zenodo_mentalhealth_df, normal_df], ignore_index=True)

In [ ]:
# See the distribution
print(data['label'].value_counts())

Preprocessing


In [ ]:
import pandas as pd
import torch
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List

# Download punkt tokenizer for sentence splitting
nltk.download("punkt")
nltk.download("punkt_tab")

# Load grammar correction model
model_name = "vennify/t5-base-grammar-correction"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Batch correction for sentences
def batch_correct(sentences: List[str], batch_size: int = 32) -> List[str]:
    corrected_sentences = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        inputs = tokenizer(
            ["gec: " + s for s in batch],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        outputs = model.generate(
            **inputs,
            max_length=512,
            num_beams=5,
            early_stopping=True
        )

        decoded = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]
        corrected_sentences.extend(decoded)

    return corrected_sentences

# Correct long texts by splitting into sentences
def correct_long_text(text: str, batch_size: int = 32) -> str:
    sentences = sent_tokenize(text)
    corrected = batch_correct(sentences, batch_size=batch_size)
    return " ".join(corrected)

# Clean an entire DataFrame
def clean_dataframe(df: pd.DataFrame, text_col: str = "text", batch_size: int = 32) -> pd.DataFrame:
    df = df.copy()
    df[text_col] = df[text_col].apply(lambda x: correct_long_text(str(x), batch_size=batch_size))
    return df

In [ ]:
final_data = clean_dataframe(data)